In [1]:
import json
import pennylane as qml
import pennylane.numpy as np

In [2]:
# angle matrix
params = np.array([[[1.0472, 0.7854, 3.1416, 0.3927],
                    [1.0472, 0.7854, 3.1416, 0.5236]],
                   [[1.0472, 0.7854, 1.5708, 0.3927],
                    [0.7854, 0.7854, 1.5708, 0.7854]]])

In [3]:
d, n, m = np.shape(params)
print(params)

[[[1.0472 0.7854 3.1416 0.3927]
  [1.0472 0.7854 3.1416 0.5236]]

 [[1.0472 0.7854 1.5708 0.3927]
  [0.7854 0.7854 1.5708 0.7854]]]


In [4]:
def W(params):
    
    """
    Subcircuit that implements the trainable block W
    
    Args:
        params (np.array): A matrix containing the parameters for the trainable block W. The length of
        params is equal to the depth of the circuit. The length of each row in params is the number 
        of qubits used. See the challenge statement for a detailed explanation
    Returns:
        Since this function is a subcircuit, you must not return anything.
    
    """

    n, m = np.shape(params)    
    for i in range(n):
        angles = params[i]
        for j in range(m):
            qml.RY(angles[j], wires=j)
        for j in range(m-1):
            qml.CNOT(wires=[j, j+1])
        qml.CNOT(wires=[m-1, 0])

In [5]:
def S(g, x, num_wires):
    
    """
    Subcircuit that implements the encoding block S
    
    Args:
        g (pennylane.Operator): A PennyLane operator representing the generator for the encoding
        gates. It must be Hermitian in order to generate a unitary. Call it as g(wires) to specify
        the wires on which it acts.
        x (complex): The scalar coefficient of the operator g.
        num_wires (int): The number of wires over which the encoding gate is broadcast.
        
        
    Returns:
        Since this function is a subcircuit, you must not return anything.
    
    """
    
    for i in range(num_wires):
        op = getattr(qml, g)
        qml.exp(op(wires=i), 1j*x)

In [6]:
dev = qml.device("default.qubit", wires=4)

@qml.qnode(dev, expansion_strategy = "device")
def quantum_model(param_set, g, x):
    
    """
    This QNode implements the quantum model with alternating trainable and encoding blocks
    
    Args:
        param_set (np.array): A numpy array whose elements are the parameter matrices for each of the trainable
        blocks. Therefore, the length of this list is equal to the number of trainable blocks, which is greater
        than, or equal to 2.
        g (pennylane.Operator): A PennyLane operator representing the generator for the encoding
        gates. It must be Hermitian in order to generate a unitary.
        x: The scalar coefficient of the operator g.
    Returns:
        (np.tensor(float)): A tensor of dimensions (2,) representing the measurement probabilities in the computational 
        basis on the first wire.
    """
    
    d, n, m = np.shape(param_set)
    
    for i in range(d-1):
        params = param_set[i]
        W(params)
        S(g, x, m)
    W(param_set[d-1])

    return qml.probs(wires=0)


In [7]:
# test 1
param_set, g, x = [[[1.0472, 0.7854, 3.1416, 0.3927],
                    [1.0472, 0.7854, 3.1416, 0.5236]],
                    [[1.0472, 0.7854, 1.5708, 0.3927],
                    [0.7854, 0.7854, 1.5708, 0.7854]]], "PauliX", 0.7854

print(np.shape(param_set))
print(param_set[0])

(2, 2, 4)
[[1.0472, 0.7854, 3.1416, 0.3927], [1.0472, 0.7854, 3.1416, 0.5236]]


In [8]:
drawer = qml.draw(quantum_model)
print(drawer(param_set, g, x))

0: ──RY(1.05)─╭●───────╭X──RY(1.05)─╭●───────╭X──Exp(0.00+0.79j X)──RY(1.05)─╭●───────╭X──RY(0.79)
1: ──RY(0.79)─╰X─╭●────│───RY(0.79)─╰X─╭●────│───Exp(0.00+0.79j X)──RY(0.79)─╰X─╭●────│───RY(0.79)
2: ──RY(3.14)────╰X─╭●─│───RY(3.14)────╰X─╭●─│───Exp(0.00+0.79j X)──RY(1.57)────╰X─╭●─│───RY(1.57)
3: ──RY(0.39)───────╰X─╰●──RY(0.52)───────╰X─╰●──Exp(0.00+0.79j X)──RY(0.39)───────╰X─╰●──RY(0.79)

──╭●───────╭X─┤  Probs
──╰X─╭●────│──┤       
─────╰X─╭●─│──┤       
────────╰X─╰●─┤       


In [9]:
expected_output = [0.46653, 0.53347]
test_output = quantum_model(param_set, g, x)
print("Expected output: ", expected_output)
print("Test output: ", test_output)

Expected output:  [0.46653, 0.53347]
Test output:  [0.46652954 0.53347046]


In [10]:
# test 2
param_set, g, x = [[[0.62832, 0.3927, 1.0472, 0.7854],
                    [0.7854, 0.31416, 0.62832, 0.5236]],
                    [[0.31416, 0.7854, 0.7854, 0.3927],
                    [0.31416, 0.3927, 0.31416, 0.3927]]], "PauliY", 0.5236

drawer = qml.draw(quantum_model)
print(drawer(param_set, g, x))

0: ──RY(0.63)─╭●───────╭X──RY(0.79)─╭●───────╭X──Exp(0.00+0.52j Y)──RY(0.31)─╭●───────╭X──RY(0.31)
1: ──RY(0.39)─╰X─╭●────│───RY(0.31)─╰X─╭●────│───Exp(0.00+0.52j Y)──RY(0.79)─╰X─╭●────│───RY(0.39)
2: ──RY(1.05)────╰X─╭●─│───RY(0.63)────╰X─╭●─│───Exp(0.00+0.52j Y)──RY(0.79)────╰X─╭●─│───RY(0.31)
3: ──RY(0.79)───────╰X─╰●──RY(0.52)───────╰X─╰●──Exp(0.00+0.52j Y)──RY(0.39)───────╰X─╰●──RY(0.39)

──╭●───────╭X─┤  Probs
──╰X─╭●────│──┤       
─────╰X─╭●─│──┤       
────────╰X─╰●─┤       


In [11]:
expected_output = [0.68594, 0.31406]
print("Expected output: ", expected_output)
print("Test output: ", quantum_model(param_set, g, x))

Expected output:  [0.68594, 0.31406]
Test output:  [0.68594115 0.31405885]
